<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo2/3_mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=10mKgunAZowpvpttdYdjoDeCM2RSMGXer" width="100%">

# Versionamiento de Modelos y Experimentos
---

En este notebook daremos una introducción práctica al versionamiento de modelos y al registro de experimentos con la herramienta `mlflow`, para ello, debemos instalarla en el entorno de Google Colaboratory. Recuerde que se trata de un sistema basado en el sistema operativo Linux, más específicamente la distribución Ubuntu:

In [ ]:
!cat /etc/os-release

Podemos instalar `mlflow` con el siguiente comando:

In [ ]:
!pip install mlflow

Adicionalmente, instalaremos algunas herramientas auxiliares:

In [ ]:
!apt install tree

Validamos que la herramienta se encuentra instalada:

In [ ]:
!mlflow --version

Finalmente, importamos las librerías necesarias:

In [ ]:
import os
import mlflow
import matplotlib.pyplot as plt
from IPython import get_ipython
from IPython.display import display

## **1. Versionamiento de Modelos y Seguimiento de Experimentos**
---

El versionamiento de modelos de aprendizaje automático (machine learning) es el proceso de llevar un registro de los diferentes modelos creados y utilizados en un proyecto de aprendizaje automático. Esto permite revertir a versiones anteriores del modelo, si es necesario, comparar diferentes versiones del modelo para ver cuál es el mejor y replicar los resultados obtenidos con una versión específica del modelo.

El seguimiento de experimentos es el proceso de registrar y rastrear los diferentes experimentos realizados en un proyecto de aprendizaje automático. Esto incluye registrar los parámetros y configuraciones utilizadas en cada experimento, así como los resultados obtenidos. El seguimiento de experimentos permite comparar diferentes experimentos y ver cómo los cambios en los parámetros y configuraciones afectan a los resultados del modelo.

Ambos, el versionamiento de modelos y el seguimiento de experimentos son fundamentales para el desarrollo de proyectos de aprendizaje automático, ya que permiten comprender mejor cómo funciona un modelo, comparar diferentes versiones y experimentos y replicar los resultados obtenidos.

Existen distintas herramientas que permiten realizar este tipo de operaciones como `dvc`, `kubeflow`, `wandb` y `mlflow`, siendo esta última la más popular para estructuración de proyectos de machine learning y seguimiento de experimentos.

## **2. MLFlow**
---

MLFlow es una plataforma open-source para la gestión de proyectos de aprendizaje automático (machine learning). Proporciona un conjunto de herramientas para facilitar el desarrollo, el seguimiento y la implementación de proyectos de aprendizaje automático.

<center><img src="https://drive.google.com/uc?export=view&id=1HC7J56QY5xZgp-8hbcscZyRFrF9Te2z6" width="50%"></center>

Algunas de las funciones principales de MLflow incluyen:

- **Seguimiento de experimentos**: MLFlow permite rastrear los experimentos realizados en un proyecto de aprendizaje automático, registrando los parámetros y configuraciones utilizadas, así como los resultados obtenidos.
- **Administración de modelos**: MLFlow permite almacenar, rastrear y desplegar modelos de aprendizaje automático.
- **Integración con diferentes herramientas y bibliotecas**: MLFlow es compatible con una variedad de herramientas y bibliotecas de aprendizaje automático populares, como TensorFlow, Keras, PyTorch, scikit-learn, entre otras.
- **Interfaz de línea de comandos y API**: MLFlow ofrece una interfaz de línea de comandos y una API en distintos lenguajes de programación para interactuar con la plataforma.

Con MLFlow podemos estructurar proyectos de machine learning sin importar la librería o el lenguaje de programación con los que fue desarrollado un modelo. Normalmente, `mlflow` cuenta con un servidor de seguimiento que se encarga de gestionar los modelos, versiones, metadatos y demás, para que equipos de científicos de datos e ingenieros de machine learning puedan fácilmente entrenar y desplegar modelos:

<img src="https://drive.google.com/uc?export=view&id=184t9zpdMmNj-TVJ7CfxAM8n4jSU8UtmK" width="80%">

MLFlow nos permite trabajar de distintas formas, incluyendo:

- Archivos locales.
- Archivos locales junto con una base de datos local en SQL.
- Servidor remoto de seguimiento, con una base de datos SQL y un backend de ejecución.

En este caso, trabajaremos con un almacenamiento local y una base de datos de sqlite por la simplicidad de configuración para un ambiente de desarrollo de Google Colab.

Comenzamos creando una carpeta donde se guardarán todos los datos de MLFLow:

In [ ]:
!mkdir mlruns

Ahora, vamos a lanzar el servidor de MLFlow utilizando una base de datos de sqlite llamada `tracking.db` y especificamos que los artefactos (archivos) serán guardados en el directorio `mlruns`:

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

Esto lanza un servidor web de MLFlow, no obstante, el servidor estará corriendo en Google Colaboratory y no en nuestro computador, por lo que no es tan fácil acceder al tablero e interactuar de forma gráfica con la herramienta.

Por esto, usaremos una herramienta conocida como [ngrok](https://ngrok.com/). Para usar esta herramienta debemos crear una cuenta gratuita y luego instalar su paquete:

In [ ]:
!pip install pyngrok

Ahora debe copiar su token de autenticación tal y como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=1RlqT7i6jSmXONmdLf6Wd8ZAHe_XYjl6H" width="50%">

Debe reemplazar el token en la siguiente variable:

In [ ]:
token = "" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Si realizó los pasos correctamente, el resultado de la celda anterior debe ser una url pública donde podrá ver el tablero de MLFlow.

> **Nota**: si está trabajando en un entorno de desarrollo local (su computador) en lugar de Google Colab, no es necesario hacer la parte de ngrok.

Adicionalmente, MLFlow crea la base de datos `tracking.db` para almacenar información del servidor. Esta almacena la siguiente información:

- `metrics`: almacena métricas de modelos.
- `model_versions`: almacena las versiones de los modelos.
- `experiments`: almacena experimentos.
- `latest_metrics`: almacena las métricas más recientes.
- `experiment_tags`: almacena etiquetas relacionadas con los experimentos.
- `tags`: almacena etiquetas globales.
- `registered_models`: almacena los modelos registrados.
- `params`: almacena hiperparámetros.
- `runs`: almacena información sobre ejecuciones.
- `registered_model_tags`: almacena etiquetas relacionadas con los modelos registrados.
- `model_version_tags`: almacena etiquetas sobre las versiones de los modelos.

Con MLFlow estaremos hablando de tres componentes:

<img src="https://drive.google.com/uc?export=view&id=1H0LOMbvvtrR3hdViTm37PCg-sme_Pqoe" width="80%">

Veamos el detalle de cada uno:

### **2.1. Tracking**
---

MLFlow nos permite hacer seguimiento de ejecuciones y experimentos en proyectos de machine learning.

<img src="https://drive.google.com/uc?export=view&id=1Y0erkQiZMd2n8nVbCajYWN2YfCh5oHwd" width="60%">

- **Ejecución (runs)**: una ejecución hace referencia al entrenamiento de un modelo de machine learning con un conjunto de hiperparámetros específicos y con determinadas métricas.
- **Experimentos (experiments)**: un experimento es el nivel de organización básico en MLFlow y nos permite agrupar varias ejecuciones, generalmente se usan para diferenciar conjuntos de datos o tipos de modelos.

El componente de seguimiento de MLFlow permite almacenar la siguiente información:

- **params**: hiperparámetros del modelo.
- **metrics**: métricas de desempeño del modelo.
- **model**: almacenar el modelo.
- **artifact**: almacenar elementos generados por el modelo.

Veamos un ejemplo con el siguiente conjunto de datos:

In [ ]:
from sklearn.datasets import make_circles
features, labels = make_circles(
    n_samples=1000,
    noise=0.1,
    factor=0.5,
    random_state=42
    )

Podemos visualizarlo:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(features[:, 0], features[:, 1], c=labels, alpha=0.5)
ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
fig.show()

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Vamos a crear un experimento en MLFlow para este conjunto de datos:

In [ ]:
exp_id = mlflow.create_experiment(name="circles", artifact_location="mlruns/")
print(exp_id)

Ahora, vamos a entrenar un modelo de regresión logística desde `sklearn`:

In [ ]:
from sklearn.linear_model import LogisticRegression

Importamos las métricas que evaluaremos de este modelo:

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

El entrenamiento de este modelo se realizará dentro de una run de MLFlow:

In [ ]:
run = mlflow.start_run(
    experiment_id = exp_id,
    run_name="default_logistic"
    )
print(run)

Entrenamos el modelo:

In [ ]:
model = LogisticRegression().fit(features, labels)

Obtenemos las predicciones del modelo:

In [ ]:
y_pred = model.predict(features)

Ahora vamos a registrar métricas de desempeño del modelo con la función `log_metrics`:

In [ ]:
mlflow.log_metrics({
    "accuracy": accuracy_score(labels, y_pred),
    "f1": f1_score(labels, y_pred),
    "precision": precision_score(labels, y_pred),
    "recall": recall_score(labels, y_pred)
    })

Vamos a almacenar el modelo desde `mlflow` con la función `log_model`

In [ ]:
mlflow.sklearn.log_model(model, "model")

Por último, vamos a crear una matriz de confusión y a almacenarla:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
fig, ax = plt.subplots()
cm = confusion_matrix(labels, y_pred)
sns.heatmap(cm, annot=True, fmt=".0f", ax=ax)
ax.set_xlabel("Predicción")
ax.set_ylabel("Real")
fig.show()
fig.savefig("confusion_matrix.png")

Guardamos la imagen dentro del almacenamiento de MLFlow:

In [ ]:
mlflow.log_artifact("confusion_matrix.png", "confusion_matrix")

Finalmente, terminamos la ejecución:

In [ ]:
mlflow.end_run()

Si actualizamos la página del tablero podremos ver que se ha creado el experimento y el run:

<img src="https://drive.google.com/uc?export=view&id=1DG5JDg6l0GEbQII6h7hLANhY4-sGKgdn" width="100%">

Si damos click sobre el nombre del run podremos ver los elementos que almacenamos como las métricas, el modelo (hablaremos de esto más adelante) y los artefactos:

<img src="https://drive.google.com/uc?export=view&id=1Xx4EouBSvAaxr0K1T1wVmaIZCrg3rpMv" width="100%">

Veamos otro ejemplo con una máquina de soporte vectorial:

In [ ]:
from sklearn.svm import SVC

Vamos a repetir el proceso con la única diferencia que ahora guardaremos hiper-parámetros del modelo con la función `log_params`. Definimos los hiperparámetros:

In [ ]:
params = {"kernel": "rbf", "C": 1.0, "gamma": 0.1}

Ahora entrenamos y almacenamos la información importante:

In [ ]:
run = mlflow.start_run(experiment_id=exp_id, run_name="svm")
model = SVC(**params).fit(features, labels)
y_pred = model.predict(features)

cm = confusion_matrix(labels, y_pred)
fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, fmt=".0f", ax=ax)
ax.set_xlabel("Predicción")
ax.set_ylabel("Real")
fig.show()
fig.savefig("confusion_matrix.png")

mlflow.log_params(params)
mlflow.sklearn.log_model(model, "model")
mlflow.log_artifact("confusion_matrix.png", "confusion_matrix")
mlflow.log_metrics({
    "accuracy": accuracy_score(labels, y_pred),
    "f1": f1_score(labels, y_pred),
    "precision": precision_score(labels, y_pred),
    "recall": recall_score(labels, y_pred)
    })
mlflow.end_run()

Desde el tablero de MLFlow podremos ver que ahora hay dos runs en este experimento:

<img src="https://drive.google.com/uc?export=view&id=1lwdaXRVYy5u6CMQKo0mMtW_LfScm_LdR" width="100%">

Si abrimos el nuevo run, podemos ver que también se han guardado los hiperparámetros del modelo.

### **2.2. Projects**
---

MLFlow se integra bastante bien con herramientas como Git o DVC, en especial, nos permite definir proyectos de ciencia de datos de forma más estructurada.

<img src="https://drive.google.com/uc?export=view&id=1_dGTZvz9pUyPWgfmD2Sz6n4yddwmQRsX" width="80%">

Vamos a agregar el siguiente script para entrenar un modelo de bosques aleatorios sobre un conjunto de datos sintético:

In [ ]:
%%writefile train.py
import sys
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_moons

params = {
        "max_depth": int(sys.argv[1]),
        "n_estimators": int(sys.argv[2])
        }
features, labels = make_moons(n_samples=1000, noise=0.1, random_state=42)

with mlflow.start_run():
    model = RandomForestClassifier(**params).fit(features, labels)
    mlflow.sklearn.log_model(model, "model")
    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy": model.score(features, labels)
        })

Creamos un nuevo experimento:

In [ ]:
exp_id = mlflow.create_experiment(name="moons", artifact_location="mlruns/")
print(exp_id)

Un proyecto de MLFlow se define a partir del archivo `MLproject`. Se trata de un archivo en formato `yaml` que define los posibles parámetros que podemos probar, sus valores por defecto, el nombre del proyecto y el comando que se debe usar para correr el script, veamos cómo definir el archivo:

In [ ]:
%%writefile MLproject
name: "mlds6"
entry_points:
    train:
        parameters:
            max_depth: {type: int, default: 2}
            n_estimators: {type: int, default: 50}
        command: "python train.py {max_depth} {n_estimators}"

Vamos a inicializar el proyecto como un repositorio de Git, recuerde identificarse:

In [ ]:
!git config --global user.email "ejemplo@unal.edu.co"
!git config --global user.name "Mi nombre o username"
!git config --global init.defaultBranch master

Inicializamos:

In [ ]:
!git init

Agregamos los dos archivos y creamos un commit:

In [ ]:
!git add train.py MLproject
!git commit -m "Proyecto de ML"

Ahora, debemos especificar al sistema cuál es la URI del servidor:

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000/"

Ahora, podemos ejecutar el script directamente desde el CLI de `mlflow`, usamos los sugientes parámetros:

- `-e`: especifica el entry point (ejecutable) definido en el archivo `MLproject`.
- `-P`: permite cambiar un parámetro del modelo.
- `--experiment-name`: permite especificar el nombre del experimento que se debe usar.
- `--env-manager`: especifica qué tipo de dependencias se usarán para la ejecución del proyecto, en este caso usamos las mismas que tiene Google Colab.
- `run-name`: nombre de la ejecución

In [ ]:
!mlflow run -e train -P max_depth=3 -P n_estimators=100 --experiment-name 'moons' --env-manager local --run-name random_forest .

Podemos entrenar nuevamente el modelo, pero con otros hiperparámetros:

In [ ]:
!mlflow run -e train -P max_depth=7 -P n_estimators=50 --experiment-name 'moons' --env-manager local --run-name random_forest .

Desde el tablero debería ver el nuevo experimento con dos runs creadas:

<img src="https://drive.google.com/uc?export=view&id=1cJOgtwdncK9W8RApUI-zkh_KRhlC-ar_" width="100%">

Si abrimos alguna de las runs, podremos ver información detallada como:

- `Run ID`: identificador de la ejecución.
- `User`: usuario que realizó la ejecución.
- `Date`: fecha en la que se realizó.
- `Duration`: tiempo que tomó el run.
- `Source`: muestra el script que generó el run.
- `Status`: muestra si el run aún está en ejecución, si terminó o tuvo errores.
- `Git commit`: identificador del commit de Git en el que se ejecutó el código.
- `Lifecycle Stage`: ciclo de vida del modelo (los veremos más en detalle más adelante).
- `Entry Point`: punto de entrada usado (especificado en el archivo MLproject).

### **2.3. Models**
---

Uno de los elementos más importantes de MLFlow es que da una forma estructurada y unificada de manejar modelos de machine learning sin importar la librería que estemos usando.

<img src="https://drive.google.com/uc?export=view&id=1yM4bW-qZu8U3MSppBSkz95RrmoJ-A8-q" width="80%">

De hecho, esto se puede ver dentro de los artefactos cuando exportamos un modelo con MLFlow:

<img src="https://drive.google.com/uc?export=view&id=1ly1mC54SBeUK6vf0iUyrozzb6MhGgv4a" width="60%">

Como podemos ver, el modelo se guarda junto con otros archivos adicionales. Dentro de estos tenemos:

- `MLmodel`: archivo en formato `yaml` que define todo lo necesario para poder reutilizar un modelo. Esto incluye: la librería de machine learning del modelo, versión de Python, versión de la librería, versión de MLFlow, función para realizar predicciones, tipo de serialización del modelo.
- `model.pkl`: modelo exportado de `sklearn`, esto puede variar en dependencia de la librería usada (más adelante veremos que esto es trasparente para el usuario).
- `conda.yaml`: dependencias de anaconda para construir el modelo.
- `python_env.yaml`: dependencias de Python para construir el modelo.
- `requirements.txt`: dependencias de `pip`.

Adicionalmente, en la parte derecha de los artefactos tenemos un botón para registrar un modelo (versionar):

<img src="https://drive.google.com/uc?export=view&id=1adgOCIiMvfkOB6mxSiZA3NnfC9heNTiJ" width="100%">

Para registrarlo debemos asignarle un nombre:

<img src="https://drive.google.com/uc?export=view&id=1IwSpld08asodRUdkgYK6o9hN8VSp0oZA" width="50%">

Con esto, veremos que el modelo será agregado a la pestaña de modelos:

<img src="https://drive.google.com/uc?export=view&id=1WfVG4c5GQChVfNze0yPGFINJksy3Ancw" width="100%">

Si damos click sobre el nombre del modelo, podremos agregar etiquetas (para luego filtrarlo), agregar una descripción o ver su versión

Por último, podemos cargar cualquier modelo del registro de MLFlow de la siguiente forma:

In [ ]:
model_name = 'moons_predictor'
model_version = 1
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
print(model)

Con este modelo podemos realizar predicciones, por ejemplo:

In [ ]:
features = [[-1.0, 1.0]]
predictions = model.predict(features)
print(predictions)

Esta forma de usar modelos es común para las librerías de machine learning soportadas por MLFlow. En las siguientes unidades seguiremos profundizando en el uso de MLFlow junto con otras herramientas.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [MLFlow Documentation](https://mlflow.org/docs/latest/index.html)
- [MLFlow Tracking](https://mlflow.org/docs/latest/tracking.html)
- [MLFlow Projects](https://mlflow.org/docs/latest/projects.html)
- [MLFlow Models](https://mlflow.org/docs/latest/models.html)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*